In [1]:
import pandas as pd
from nltk.stem import PorterStemmer
import string

# Esercizio 1.1 - Similarity

Calcolo della similarità fra i termini del file *defs.csv* .

Caricamento del file *defs.csv* .

In [2]:
df_defs = pd.read_csv(filepath_or_buffer="utils/defs.csv", index_col=0).dropna()
df_defs.reset_index(drop=True, inplace=True)
NUM_DEFS = len(df_defs) # 30
df_defs.head()

,Courage,Paper,Apprehension,Sharpener
0,property that allows you to face any situation...,"cellulose material that can be cut, folded and...","something strange, which causes a strange feel...",tool equipped with a blade that allows you to ...
1,Ability to face our own fears and do something...,Material derived from trees and used in sever...,fearful expectation or anticipation,Object used to shapen a pencil
2,the ability to face thing without fear,a type of material made from cellulose,A mood where one feel agitation,An object to sharpen a pencil
3,Inner strength that allow you to face particul...,Product obtained from wood cellulose. It is us...,State of disturbance,Tool used to sharpen pencils
4,Ability to control the fear,Flat material made from wood used for writing,Worry about the future,Little object which allow to sharpen a pencil


Fase di preprocessing (_lowercasing_, _punctuation removal_, _stopword removal_, _stemming_).

In [3]:
# Loading stopwords list from file
stopwords = []
for line in open("utils/stop_words_FULL.txt", 'r').readlines():
    stopwords.append(line.rstrip('\n'))
stopwords = pd.Series(stopwords)

# Initializing nltk.PorterStemmer()
ps = PorterStemmer()

for c in df_defs.columns:
    # Lowercasing
    df_defs[c] = df_defs[c].str.lower()
    # Punct removal
    tmp = df_defs[c].apply(lambda x: str(x).translate(str.maketrans('', '', string.punctuation)))
    df_defs[c] = tmp
    # Stopword removal
    tmp = df_defs[c].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stopwords.values]))
    df_defs[c] = tmp
    # Stemming
    tmp = df_defs[c].apply(lambda x: ' '.join([ps.stem(word) for word in str(x).split()]))
    df_defs[c] = tmp
df_defs.head()

,Courage,Paper,Apprehension,Sharpener
0,properti allow face situat despit feel fear,cellulos materi cut fold written,strang strang feel strang normal abnorm,tool equip blade allow sharpen pencil
1,abil face fear scar unpleas,materi deriv tree context,fear expect anticip,object shapen pencil
2,abil face thing fear,type materi cellulos,mood feel agit,object sharpen pencil
3,inner strength allow face situat,product wood cellulos write,disturb,tool sharpen pencil
4,abil control fear,flat materi wood write,worri futur,object allow sharpen pencil


Calcolo della similarità fra le definizioni di ogni termine.

La similarità viene calcolata come la media fra i valori medi delle _Bag of Words_ calcolati su tutte le combinazioni di definizioni. Ogni definizione avrà un certo numero medio di parole che sono presenti anche in altre definizioni, facendo la media fra tutti i valori si ottiene il punteggio di similarità del termine.

Ci aspettiamo che i termini __concreti__ ('*Paper*', '*Sharpener*') abbiamo un valore di similarità maggiore rispetto a quelli __astratti__ ('*Courage*', '*Apprehension*')

In [4]:
def bag_of_words(d, defs):
    """
    Find the average intersection length between a definition and the others.

    :param d: the definition
    :param defs: the other definitions

    :return: average length of the intersection
    """
    d = set(d.split())
    sum = 0
    for other_d in defs:
        bow = d.intersection(other_d.split()) # calcolo intersezione fra le due definizioni
        sum += len(bow) / min(len(d), len(other_d.split())) # normalizzazione
    return sum/NUM_DEFS


def compute_sim(c):
    """
    Find the average bow value between all the definitions of a term.

    :param c: the term's column in the dataframe

    :return: average bow value
    """
    sum = 0
    for i in range(NUM_DEFS):
        sum += bag_of_words(df_defs[c][i], df_defs[c]) # calcoliamo la similarità di ogni definizione con a tutte le altre
    return sum/NUM_DEFS # facciamo la media


df_res = pd.DataFrame(index=['Generico', 'Specifico'], columns=['Astratto', 'Concreto'])
df_res_terms = pd.DataFrame(index=['Generico', 'Specifico'], columns=['Astratto', 'Concreto'])
for c, i, j in zip(df_defs.columns, [0,0,1,1], [0,1,0,1]):
    df_res_terms.iloc[i,j] = f'{c}: {compute_sim(c)}'
    df_res.iloc[i,j] = compute_sim(c)
    # media delle medie delle intersezioni fra tutte le combinazioni di definizioni della colonna c
    # per ogni definizione calcolo la bow media con tutte le altre defs, faccio la media delle medie ed ho la similarità
print('--- Valori di similarità dei termini ---')
print(df_res_terms, '\n')
print('--- Valore medio lungo le colonne ---')
print(df_res.mean(axis=0), '\n')
print('--- Valore medio lungo le righe ---')
print(df_res.mean(axis=1))

--- Valori di similarità dei termini ---
                                    Astratto                       Concreto
Generico        Courage: 0.32249735449735445     Paper: 0.37825925925925924
Specifico  Apprehension: 0.15857407407407406  Sharpener: 0.5051111111111111 

--- Valore medio lungo le colonne ---
Astratto    0.240536
Concreto    0.441685
dtype: float64 

--- Valore medio lungo le righe ---
Generico     0.350378
Specifico    0.331843
dtype: float64


I risultati ottenuti confermano le ipotesi iniziali, in particolare:
- se andiamo dall'__astratto al concreto__ abbiamo un valore __crescente__ di similarità
- se andiamo dal __generico allo specifico__ abbiamo un valore __decrescente__ di similarità


# Esercizio 1.2 - Similarity explanation
Spiegazione della similarità usando una lista delle parole più frequentemente usate nelle definizioni.

Per ogni termine viene calcolato:
 - il numero totale di paorle diverse usate nelle definizioni
 - una lista delle parole maggiormente usate (occorrono in almeno la metà delle definizioni)

In [5]:
for c in df_defs.columns:
    words_count = df_defs[c].str.split(expand=True).stack().value_counts() # conta le occorrenze di ogni parola lungo tutta la colonna del termine
    frequent_words = words_count[words_count > 15]  # prendiamo le parole che occorrono in 1/2 (15) delle definizioni, supponendo che ogni parola compaia al più una volta in ogni definizione
    print(f'Word: \'{c}\'\n'
          f'Number of words: {len(words_count)}\n'
          f'List of frequent words: \n{frequent_words}\n')

Word: 'Courage'
Number of words: 51
List of frequent words: 
abil    19
fear    18
dtype: int64

Word: 'Paper'
Number of words: 50
List of frequent words: 
materi    23
write     16
dtype: int64

Word: 'Apprehension'
Number of words: 57
List of frequent words: 
Series([], dtype: int64)

Word: 'Sharpener'
Number of words: 28
List of frequent words: 
pencil     25
sharpen    19
tool       16
dtype: int64



Dai dati ottenuti notiamo come per i termini '*Courage*', '*Paper*' ed '*Apprehension*' il numero delle parole usato sia molto alto, sintomo di una difficoltà nell'individuarli correttamente.

Per '*Apprehension*' abbiamo il valore maggiore (57) ed infatti è il termine con il valore di similarità più basso. Questo rispecchia ciò che si era affermato in precedenza, infatti, essendo un termine astratto, è molto difficile da descrivere ed ognuno tende ad avere una propria rappresentazione.

Discorso opposto per '*Sharpener*', il quale ha invece poche parole usate (28) ed è quello con la similarità maggiore. Essendo un oggetto fisico è più semplice descriverlo e si tende ad usare un linguaggio più uniforme.

Fra le parole più usate nelle definizioni possiamo osservare che per gli oggetti __concreti__ si ha una concentrazione maggiore sugli stessi termini, in quanto le caratteristiche sono visibili ed è semplice descriverle.
Questo non vale per gli oggetti __astratti__ i quali hanno una distribuzione delle parole diversa, basti guardare ad '*Apprehension*' che non ha nessuna parola frequente.